In [3]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel-ner-qa'
model_name = "refuel-llm"
label_column = 'Keyword_answer'
text_column = 'context'
example_selection_label_column = 'has_Keyword'
label_symbol = "^^^^"
few_shot_num = 8
few_shot_selection = "semantic_similarity"
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=False
temperature=0.1
random_shuffle_examples = True
random_shuffle_examples_seed = 1

In [2]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION 
import sys

In [ ]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [4]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [5]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [6]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [7]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
label_type = label_column.split('_')[0]
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_' + label_type + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [8]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name.split('/')[1]}"

In [9]:
sym_len = len(label_symbol)
label_prefix, label_suffix = label_symbol[:sym_len//2], label_symbol[sym_len//2:]

label_description = {"Food": "are specific food item that caused the potential foodborne illness incident.",
                     "Symptom": "are specific symptoms experienced by the affected person as a result of the suspected foodborne illness.",
                     "Location": "pretrain to the location where the affected person purchased or acquired the food associated with the potential foodborne illness.",
                     "Keyword": """include other relevant keywords or terms associated with foodborne illnesses, such as "food poisoning"."""}

task_guideline = (f"You are an expert at extracting {label_type} entities that are related to foodborne illness incident from text. " 
                  f"In the given text, your task is to label {label_type} entities that {label_description[label_type]}. "
                  f"""Note that you should surround the extracted entities in the text with "{label_prefix}" and "{label_suffix}". """
                  "Use the following examples as a guide for your predictions and format your responses similarly.")

In [10]:
test_data = pd.read_csv(data_path)

In [11]:
config = {
    "task_name": task_name,
    "task_type": "question_answering",
    "dataset": {
        "label_column": label_column,
        "text_column": text_column,
        "example_selection_label_column": example_selection_label_column,
        "delimiter": ",",
        "label_description": label_description 
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": task_guideline,
        "output_guidelines": f"You will answer with just the correct labeled sentence and nothing else. Note that if the given text does not include any {label_type} entity related to foodborne illness, you will answer with the input text.",
        "labels": [
            "Food",
            "Symptom",
            "Location",
            "Keyword"
        ],
        "few_shot_examples": few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Input: {{{text_column}}}\nOutput: {{{label_column}}}"
    }
}

config = NewAutoLabelConfig(config)

In [12]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-01-30 23:53:09 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.


In [13]:
ds = AutolabelDataset(test_data[[text_column, example_selection_label_column, label_column]], config=config)

In [14]:
agent.plan(ds)

2024-01-30 23:53:12 sentence_transformers.SentenceTransformer INFO: Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2024-01-30 23:53:14 sentence_transformers.SentenceTransformer INFO: Use pytorch device: cuda


Batches:   0%|          | 0/78 [00:00<?, ?it/s]

Output()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
# now, do the actual labeling
ds = agent.run(ds)

Output()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

2024-01-30 23:57:17 autolabel.models.refuel WARNING: Retrying autolabel.models.refuel.RefuelLLM._label_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://llm.refuel.ai/models/refuel-llm/generate.
2024-01-30 23:57:19 autolabel.models.refuel WARNING: Retrying autolabel.models.refuel.RefuelLLM._label_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://llm.refuel.ai/models/refuel-llm/generate.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-30 23:57:23 autolabel.models.refuel WARNING: Retrying autolabel.models.refuel.RefuelLLM._label_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://llm.refuel.ai/models/refuel-llm/generate.
2024-01-30 23:57:25 autolabel.models.refuel WARNING: Retrying autolabel.models.refuel.RefuelLLM._label_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://llm.refuel.ai/models/refuel-llm/generate.
2024-01-30 23:57:27 autolabel.models.refuel WARNING: Retrying autolabel.models.refuel.RefuelLLM._label_with_retry in 4.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://llm.refuel.ai/models/refuel-llm/generate.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-30 23:57:34 autolabel.models.refuel WARNING: Retrying autolabel.models.refuel.RefuelLLM._label_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://llm.refuel.ai/models/refuel-llm/generate.
2024-01-30 23:57:36 autolabel.models.refuel WARNING: Retrying autolabel.models.refuel.RefuelLLM._label_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://llm.refuel.ai/models/refuel-llm/generate.


In [19]:
metrics = ds.eval()

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃ f1     ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0.6      │ 10      │ 1.0             │ 0.9264 │
└──────────┴─────────┴─────────────────┴────────┘

In [17]:
ds.df.to_csv(output_path, index=False)
ds.df.to_pickle(output_path.replace('.csv', '.pkl'))